# Autocombat TOP and StrokeMRI vs. EDIS harmonized datasets

Note this must be run in the `comscan6` environment

## import libraries

In [1]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har
import cvasl.vendor.comscan.neurocombat as autocombat

C:\Users\CandaceMoore\miniconda3\envs\comscan6\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [2]:
# Datasets for this work
MRI_path = '../our_datasets/StrokeMRI/'
TOP_path = '../our_datasets/TOP/'
EDIS_path = '../our_datasets/EDIS/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
EDIS_file = os.path.join(EDIS_path, file_name)

TOP = pd.read_csv(TOP_file,index_col=0)
MRI = pd.read_csv(MRI_file,index_col=0)
EDIS = pd.read_csv(EDIS_file,index_col=0)

In [3]:
sex_mapping = {1:0,2:1}
EDIS = EDIS.assign(Sex = EDIS.Sex.map(sex_mapping))
EDIS.head(3)

,participant_id,ID,Age,Sex,Site,GM_vol,WM_vol,CSF_vol,GM_ICVRatio,GMWM_ICVRatio,WMHvol_WMvol,WMH_count,ACA_B_CoV,MCA_B_CoV,PCA_B_CoV,TotalGM_B_CoV,ACA_B_CBF,MCA_B_CBF,PCA_B_CBF,TotalGM_B_CBF
0,sub-ED003_1,1,66,1,1,0.55942,0.43780,0.24159,0.45158,0.80498,0.009468,23,0.60989,0.66494,0.73127,0.68457,135.4218,142.7780,117.9492,124.4947
1,sub-ED005_1,2,71,0,1,0.60938,0.51911,0.28142,0.43221,0.80040,0.007386,26,0.52744,0.68096,0.69366,0.66570,191.8253,192.3582,152.7199,166.6538
2,sub-ED009_1,3,84,1,1,0.52308,0.39046,0.31412,0.42608,0.74413,0.074138,45,0.67113,0.67629,0.74404,0.68511,87.4015,93.3342,79.0872,81.7578


In [4]:
TOP = TOP.drop([ 'ID'], axis= 1)
TOP.head(3)

,participant_id,Age,Sex,Site,GM_vol,WM_vol,CSF_vol,GM_ICVRatio,GMWM_ICVRatio,WMHvol_WMvol,WMH_count,ACA_B_CoV,MCA_B_CoV,PCA_B_CoV,TotalGM_B_CoV,ACA_B_CBF,MCA_B_CBF,PCA_B_CBF,TotalGM_B_CBF
0,sub-0668_1,50.40,1,1,0.64252,0.48470,0.26711,0.46081,0.80843,0.004506,18,0.17747,0.20996,0.24951,0.24191,90.5294,80.9975,57.9326,72.1759
1,sub-0532_1,37.02,1,1,0.71890,0.59546,0.31236,0.44193,0.80798,0.002501,24,0.17620,0.23222,0.21187,0.23518,101.2955,90.3336,67.8547,80.1958
2,sub-0529_1,30.57,1,1,0.66610,0.45798,0.28326,0.47330,0.79873,0.005144,20,0.19233,0.20855,0.19810,0.21863,108.6481,99.6353,72.4565,87.2374


In [5]:
MRI = MRI.drop(['ID'],axis = 1)
MRI.tail(5)

,participant_id,Age,Sex,Site,GM_vol,WM_vol,CSF_vol,GM_ICVRatio,GMWM_ICVRatio,WMHvol_WMvol,WMH_count,ACA_B_CoV,MCA_B_CoV,PCA_B_CoV,TotalGM_B_CoV,ACA_B_CBF,MCA_B_CBF,PCA_B_CBF,TotalGM_B_CBF
573,sub-5928501_1,67.178082,1,1,0.57040,0.44852,0.33133,0.42244,0.75462,0.013977,30,0.18116,0.20147,0.23738,0.24576,72.0298,60.8287,43.0370,50.3654
574,sub-5910502_1,67.794521,1,1,0.56474,0.44446,0.34731,0.41632,0.74397,0.007008,32,0.19077,0.21919,0.24733,0.23882,69.3495,60.6663,52.6914,54.0085
575,sub-5931002_1,54.769863,0,1,0.66914,0.56129,0.41598,0.40642,0.74734,0.015069,40,0.25569,0.26058,0.25183,0.27917,68.9781,58.6687,46.1167,53.3346
576,sub-5925701_1,66.639344,1,1,0.58411,0.47607,0.34966,0.41431,0.75199,0.011519,30,0.17745,0.26865,0.30003,0.28667,59.8056,58.3224,40.4508,46.3041
577,sub-5934201_1,45.336986,1,1,0.62603,0.44826,0.30040,0.45540,0.78148,0.004756,14,0.16992,0.20050,0.20542,0.19957,97.4488,84.5106,74.9190,81.1740


In [6]:
EDIS = EDIS.drop(['ID'],axis = 1)
EDIS.tail(5)

,participant_id,Age,Sex,Site,GM_vol,WM_vol,CSF_vol,GM_ICVRatio,GMWM_ICVRatio,WMHvol_WMvol,WMH_count,ACA_B_CoV,MCA_B_CoV,PCA_B_CoV,TotalGM_B_CoV,ACA_B_CBF,MCA_B_CBF,PCA_B_CBF,TotalGM_B_CBF
341,sub-ED975_1,68,0,1,0.51530,0.41569,0.36956,0.39622,0.71584,0.019204,19,0.38529,0.39935,0.44822,0.41982,99.0492,99.9323,73.8232,86.9992
342,sub-ED976_1,62,1,1,0.59745,0.50880,0.26282,0.43639,0.80803,0.002932,24,0.53279,0.44099,0.68616,0.55345,77.5146,89.7554,55.4640,72.3751
343,sub-ED991_1,68,0,1,0.49445,0.40824,0.40572,0.37790,0.68991,0.004157,23,0.73732,0.70174,0.81250,0.76772,56.0931,68.0910,55.1070,54.1563
344,sub-ED994_1,71,0,1,0.53310,0.47591,0.42071,0.37287,0.70574,0.018577,16,0.71091,0.83803,1.02090,0.85908,93.2826,92.9924,69.0299,79.0787
345,sub-ED999_1,60,1,1,0.50500,0.38452,0.27453,0.43383,0.76416,0.001532,12,0.49046,0.48276,0.60128,0.51331,91.0541,90.0715,73.9869,83.2005


In [7]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI['Site'] = 0
TOPMRI.head(3) 

,participant_id,Age,Sex,Site,GM_vol,WM_vol,CSF_vol,GM_ICVRatio,GMWM_ICVRatio,WMHvol_WMvol,WMH_count,ACA_B_CoV,MCA_B_CoV,PCA_B_CoV,TotalGM_B_CoV,ACA_B_CBF,MCA_B_CBF,PCA_B_CBF,TotalGM_B_CBF
0,sub-0668_1,50.40,1,0,0.64252,0.48470,0.26711,0.46081,0.80843,0.004506,18,0.17747,0.20996,0.24951,0.24191,90.5294,80.9975,57.9326,72.1759
1,sub-0532_1,37.02,1,0,0.71890,0.59546,0.31236,0.44193,0.80798,0.002501,24,0.17620,0.23222,0.21187,0.23518,101.2955,90.3336,67.8547,80.1958
2,sub-0529_1,30.57,1,0,0.66610,0.45798,0.28326,0.47330,0.79873,0.005144,20,0.19233,0.20855,0.19810,0.21863,108.6481,99.6353,72.4565,87.2374


In [8]:
TOPMRI.columns = TOPMRI.columns.str.lower()

In [9]:
EDIS.columns = EDIS.columns.str.lower()

In [15]:
TOPMRIEDIS = pd.concat([TOPMRI, EDIS])
TOPMRIEDIS

,participant_id,age,sex,site,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,wmhvol_wmvol,wmh_count,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf
0,sub-0668_1,50.40,1,0,0.64252,0.48470,0.26711,0.46081,0.80843,0.004506,18,0.17747,0.20996,0.24951,0.24191,90.5294,80.9975,57.9326,72.1759
1,sub-0532_1,37.02,1,0,0.71890,0.59546,0.31236,0.44193,0.80798,0.002501,24,0.17620,0.23222,0.21187,0.23518,101.2955,90.3336,67.8547,80.1958
2,sub-0529_1,30.57,1,0,0.66610,0.45798,0.28326,0.47330,0.79873,0.005144,20,0.19233,0.20855,0.19810,0.21863,108.6481,99.6353,72.4565,87.2374
3,sub-0393_1,47.05,1,0,0.65366,0.50449,0.22497,0.47260,0.83735,0.006571,27,0.18299,0.22545,0.20645,0.24245,92.8599,81.2144,61.5686,73.7697
4,sub-0593_1,44.63,1,0,0.66182,0.48806,0.27842,0.46336,0.80507,0.005731,35,0.18738,0.19636,0.20306,0.20339,83.5410,77.6834,67.6360,71.2309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,sub-ED975_1,68.00,0,1,0.51530,0.41569,0.36956,0.39622,0.71584,0.019204,19,0.38529,0.39935,0.44822,0.41982,99.0492,99.9323,73.8232,86.9992
342,sub-ED976_1,62.00,1,1,0.59745,0.50880,0.26282,0.43639,0.80803,0.002932,24,0.53279,0.44099,0.68616,0.55345,77.5146,89.7554,55.4640,72.3751
343,sub-ED991_1,68.00,0,1,0.49445,0.40824,0.40572,0.37790,0.68991,0.004157,23,0.73732,0.70174,0.81250,0.76772,56.0931,68.0910,55.1070,54.1563
344,sub-ED994_1,71.00,0,1,0.53310,0.47591,0.42071,0.37287,0.70574,0.018577,16,0.71091,0.83803,1.02090,0.85908,93.2826,92.9924,69.0299,79.0787


In [16]:
TOPMRIEDIS['decade']=(TOPMRIEDIS['age']/10).round()
#TOPMRI['decade']

In [17]:
TOPMRIEDIS = TOPMRIEDIS.sort_values(by='age')
TOPMRIEDIS.reset_index(inplace=True)
TOPMRIEDIS['fine_grain'] = TOPMRIEDIS['age'].rolling(2).sum()/2
TOPMRIEDIS

,index,participant_id,age,sex,site,gm_vol,wm_vol,csf_vol,gm_icvratio,gmwm_icvratio,...,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,aca_b_cbf,mca_b_cbf,pca_b_cbf,totalgm_b_cbf,decade,fine_grain
0,481,sub-0386_1,17.720000,0,0,0.74568,0.51859,0.27316,0.48502,0.82233,...,0.16566,0.21039,0.22326,0.22553,92.3001,89.2040,67.6051,77.1479,2.0,NaN
1,110,sub-0337_1,18.150000,1,0,0.71129,0.44216,0.18840,0.53008,0.85960,...,0.18783,0.20972,0.19325,0.21806,96.9223,90.6592,67.5734,82.0479,2.0,17.935000
2,269,sub-0239_1,18.400000,0,0,0.69937,0.46534,0.24350,0.49664,0.82709,...,0.19085,0.23348,0.17550,0.24048,110.3567,106.6904,75.3278,91.4069,2.0,18.275000
3,163,sub-0628_1,18.490000,0,0,0.87101,0.53531,0.23551,0.53051,0.85656,...,0.19333,0.22139,0.15207,0.21070,83.9998,77.2528,58.1547,71.0279,2.0,18.445000
4,81,sub-0572_1,18.520000,0,0,0.82831,0.54979,0.26062,0.50546,0.84096,...,0.19975,0.23284,0.20153,0.24734,81.3364,80.0378,54.2705,68.6422,2.0,18.505000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,201,sub-5911602_1,85.776832,1,0,0.50250,0.33882,0.40174,0.40424,0.67681,...,0.20391,0.19409,0.20740,0.23931,61.1184,52.5087,40.8758,48.2727,9.0,85.388416
1449,334,sub-5937602_1,85.816139,0,0,0.65526,0.49287,0.51782,0.39333,0.68917,...,0.20419,0.22845,0.20043,0.24442,75.5130,66.7933,48.1881,57.7182,9.0,85.796486
1450,438,sub-5922001_1,88.825294,0,0,0.53368,0.41073,0.52076,0.36424,0.64457,...,0.18378,0.19728,0.18157,0.18748,109.7799,95.2840,91.5285,92.8757,9.0,87.320716
1451,384,sub-5942301_1,92.049315,0,0,0.48348,0.40179,0.55615,0.33542,0.61417,...,0.21289,0.23416,0.24252,0.27414,78.1060,63.2504,59.7786,57.7572,9.0,90.437304


In [18]:
TOPMRIEDIS[2:].fine_grain.iloc[::2] = TOPMRIEDIS[:].fine_grain.iloc[1::2]
#TOPMRI['fine_grain']

C:\Users\CandaceMoore\miniconda3\envs\comscan6\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [19]:
TOPMRIEDIS['fine_grain'][0] = TOPMRIEDIS['fine_grain'][1]
TOPMRIEDIS['fine_grain']

C:\Users\CandaceMoore\miniconda3\envs\comscan6\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0       17.935000
1       17.935000
2       17.935000
3       18.445000
4       18.445000
          ...    
1448    84.936986
1449    85.796486
1450    85.796486
1451    90.437304
1452    90.437304
Name: fine_grain, Length: 1453, dtype: float64

In [ ]:
TOPMRIEDIS['decade'].unique()

In [ ]:
TOPMRIEDIS.columns

In [ ]:
combat = autocombat.Combat(
    features=[ 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
        'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
        'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
    sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['decade'],)

fg_combat = autocombat.Combat(
    features=[ 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
    sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['fine_grain'],)


In [ ]:
print(combat.fit(TOPMRIEDIS))
print(fg_combat.fit(TOPMRIEDIS))

In [ ]:
print(combat.gamma_star_)
print(fg_combat.gamma_star_)

In [ ]:
transformed_TOPMRIEDIS= combat.transform(TOPMRIEDIS)
transformed_TOPMRIEDIS.head(3)

In [ ]:
fg_transformed_TOPMRIEDIS= fg_combat.transform(TOPMRIEDIS)
fg_transformed_TOPMRIEDIS.head(3)

In [ ]:
TOPMRIEDIS.head(3)

In [ ]:
transformed_TOPMRIEDIS= combat.transform(TOPMRIEDIS)
transformed_TOPMRIEDIS.head(3)

In [ ]:
fg_transformed_TOPMRIEDIS= fg_combat.transform(TOPMRIEDIS)
fg_transformed_TOPMRIEDIS.head(3)

In [ ]:
transformed_TOPMRIEDIS.columns

In [ ]:
TOPMRI_transformed = transformed_TOPMRIEDIS[ transformed_TOPMRIEDIS['site']==0]
TOPMRI_transformed = TOPMRI_transformed.drop(['site', 'decade','fine_grain', 'index'], axis=1)
TOPMRI_transformed.head(3)

In [ ]:
fg_TOPMRI_transformed = fg_transformed_TOPMRIEDIS[ fg_transformed_TOPMRIEDIS['site']==0]
fg_TOPMRI_transformed = fg_TOPMRI_transformed .drop(['site', 'decade','fine_grain', 'index'], axis=1)
fg_TOPMRI_transformed.head(3)

In [ ]:
EDIS_transformed = transformed_TOPMRIEDIS[ transformed_TOPMRIEDIS['site']==1]
EDIS_transformed = EDIS_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
EDIS_transformed.head(3)

In [ ]:
fg_EDIS_transformed = fg_transformed_TOPMRIEDIS[fg_transformed_TOPMRIEDIS['site']==1]
fg_EDIS_transformed = fg_EDIS_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
fg_EDIS_transformed.head(3)

In [ ]:
TOPMRI_transformed.to_csv('harm_results/autocombat/autocom_harm_topmri_v_e.csv')
EDIS_transformed.to_csv('harm_results/autocombat/autocom_harm_EDIS.csv')

fg_TOPMRI_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_topmri_v_e.csv')
fg_EDIS_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_EDIS1.csv')

# Data quality check

In [ ]:
TOPMRI_transformed.isna().sum().sum()

In [ ]:
EDIS_transformed.isna().sum().sum()

In [ ]:
number_columns = ['sex', 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
        'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',]

In [ ]:
#neuro_harm_top.head(3)
(TOPMRI_transformed[number_columns] < 0).sum()

In [ ]:
(EDIS_transformed[number_columns] < 0).sum()

In [ ]:
EDIS_transformed

## So some variables are again problematic, becuas they are negative

 We will wait about logging it until we see other columns we may want to log across all datasets